In [1]:
import re

import ipywidgets as widgets
import pandas as pd
import requests
from IPython.display import display
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [2]:
base_url = "https://covid19br.s3-sa-east-1.amazonaws.com"

In [3]:
r = requests.get(f"{base_url}/?delimiter=/&prefix=pkl/")

ConnectionError: HTTPSConnectionPool(host='covid19br.s3-sa-east-1.amazonaws.com', port=443): Max retries exceeded with url: /?delimiter=/&prefix=pkl/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020C3A9D2AB0>: Failed to resolve 'covid19br.s3-sa-east-1.amazonaws.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
filenames = re.findall(r"<Key>(.+?)</Key>", r.content.decode())
filenames = [name for name in filenames if name.endswith("pkl")]

In [ ]:
def month_number(month: str):
    months = ["jan", "fev", "mar", "abr", "mai", "jun", "jul", "ago", "set", "out", "nov", "dez"]
    convert = {j: i+1 for i, j in enumerate(months)}
    return convert[month]

def filename_date(name : str):
    short = re.match(r"^.*COVIDBR_(.*)\.xlsx\.pkl$", name).group(1)
    day = int(short[:2])
    month = month_number(short[2:5])
    year = int(short[5:])
    return pd.Timestamp(year=year, month=month, day=day)

def date_repr(date):
    return "{:02d}/{:02d}/{}".format(date.day, date.month, date.year)

In [ ]:
filenames = sorted(filenames, key=lambda x: filename_date(x))
options = [date_repr(filename_date(f)) for f in filenames]

In [ ]:
# In pickle format for fast loading
list(reversed(filenames))

[]

In [ ]:
df_map = {}
for filename, opt in tqdm(list(zip(filenames, options))):
    url = f"{base_url}/{filename}"
    df_map[opt] = pd.read_pickle(url)

0it [00:00, ?it/s]

In [ ]:
print(filenames, len(filenames))


[] 0


In [ ]:
if len(filenames) > 0:
    date_slider = widgets.SelectionSlider(
        options=[
            (date_repr(filename_date(f)), filename_date(f))
            for f in filenames
        ],
        value=filename_date(filenames[-1]),
        description='File date:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
else:
    date_slider = widgets.SelectionSlider(
        options=['Nenhum arquivo encontrado'],
        value='Nenhum arquivo encontrado',
        description='File date:',
        disabled=True,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )

display(date_slider)


SelectionSlider(continuous_update=False, description='File date:', disabled=True, options=('Nenhum arquivo enc…